# Lesson 06 - Introduction to RDDs

## The Resilient Distributed Dataset

The primary data abstraction in Apache Spark is the **Resilient Distributed Dataset**, or **RDD**. According to the [Apache Spark documentation](https://spark.apache.org/docs/latest/rdd-programming-guide.html#resilient-distributed-datasets-rdds), an RDD is a "fault-tolerant collection of elements that can be operated on in parallel". RDDs are superficially similar to the `list` data type that you may already be familiar with from Python. However, there are several important differences between lists and RDDs. To better understand the important features of an RDD, we will decompose the name of the data type, taking the words in reverse order. 

* **Dataset.** An RDD is a collection of values. An individual value contained within an RDD is called an **element** of the RDD. The elements of an RDD can be of any data type and a single RDD can contain elements of many different data types. 
* **Distributed.** When we create an RDD, it is split into several smaller pieces called **partitions**. Spark distributes the partitions of the RDD across the nodes in the cluster. 
* **Resilient.** An RDD is considered "fault-tolerant". An RDD is generally able to be fully reconstructed if some of its partitions are damaged or lost as a result of node failures. 

Additionally, it should be noted that RDDs are immutable data types. This means that once created, the contents of an RDD cannot be altered. If we apply a transformation to the contents of an RDD (such as squaring each element), a new RDD is created containing the results of the transformation.

## Low-Level Data Structure

Spark RDDs are low-level data structures on top of which all higher-level Spark data structures, such as DataFrames, are constructed. In Spark version 1.x, RDDs were the primary data type in Spark. This changed in version 2.x, at which point Spark SQL became the foremost component of Spark and DataFrames became the primary data type. 

Today, DataFrames are used much more frequently than RDDs. The DataFrame API in Spark is more highly developed and provides beneath-the-hood optimization techniques that are not available for RDDs. However, there are still times when one might wish to use RDDs, as they allow for more control and customization than DataFrames.

## Transformations, Actions, and Lazy Evaluation

RDDs come equipped with several methods, which can be grouped into two categories: **transformations** and **actions**. 

* A **transformation** is a method that applies some sort of operation to the elements of an RDD returning a new, transformed RDD. 
* An **action** is a method that represents some sort of calculation in which information is returned to the driver. 

In some sense, the core distinction between these two types of operations is that a transformation returns a new RDD, while an action produces output that is not an RDD. For example, an action might display output to the screen, write data to disk, or return a different data type, such as a Python list. 

Data processing tasks in Spark are performed according to a strategy know as **lazy evaluation** in which tasks are not performed immediately, but are instead postponed until they are required in order to fulfill a request for some specific output. 

When we call a transformation in Spark, the resulting RDD is not calculated immediately. Instead, the requested calculation is put into a queue, only to be performed when we call an action on the resulting RDD. This evaluation strategy had several benefits. It allows Spark to delay expensive calculations until absolutely necessary. It also allows Spark to optimize a sequence of transformations by combining similar transformations together, or eliminating redundant transformations. We will discuss lazy evaluation in more detail in a later lecture.

## The SparkContext

The **`SparkContext`** object provides an API for creating and working with RDDs. There are multiple ways to create a `SparkContext`, but one is created automatically when we create an instance of the `SparkSession`. In the code cell below, we create a `SparkSession` object and then assign the associated `SparkContext` object to a variable named `sc`.

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

We can print the `SparkContext` object to get information about our Spark environment. The `SparkContext` also contains a `version` attribute that stores the version of Spark that we are currently running.

In [0]:
print(sc)
print(sc.version)

<SparkContext master=spark://10.215.242.170:7077 appName=Databricks Shell>
3.0.1

## Creating RDDs

There are two common ways to create RDDS:
1. Using the `parallelize()` method of the `SparkContext` object to create an RDD from an in-memory object, such as a list or NumPy array. 
2. Using the `textFile()` method of the `SparkContext` object to create an RDD from a data file.

We will discuss reading files from external sources later in this lesson. For now, we will focus on using `parallelize` to create RDDs from in-memory collections. To illustrate this process, we will begin by creating a NumPy array consisting of 50 randomly generated integers.

In [0]:
import numpy as np
random_array = np.random.choice(range(100), size=50)

In the cell below, we will use the `parallelize()` method to create an RDD from our newly created array. We will also print the type of the RDD to confirm that it is in fact an RDD object.

In [0]:
random_rdd = sc.parallelize(random_array)
print(type(random_rdd))

<class 'pyspark.rdd.RDD'>

## The collect() Action

Suppose that we would like to view the contents of an RDD. It would be natural to try to place the RDD inside of the `print()` function. Let's see what happens if we do this.

In [0]:
print(random_rdd)

ParallelCollectionRDD[0] at readRDDFromInputStream at PythonRDD.scala:413

Notice that the output displayed for the cell is not a squence of 50 random integers. Instead we see some technical information regarding the way the RDD was created. 

The reason why `print(random_rdd)` did not print the contents of the RDD is that the RDD does not actually contain any information yet. As mentioned above, RDDs are evaluated lazily. This means that the contents of an RDD are not actually generated until we call an action that requires those values. The Python `print()` function is not a Spark action. 

We can force the contents of an RDD to be generated by using the `collect()` action. This RDD method tells Spark to calculate the contents of the RDD and then return those contents to the driver in the form of a list. 

In the cell below, we call the `collect()` action on `random_rdd`, and then print the resulting list.

In [0]:
print(random_rdd.collect())

[30, 67, 47, 38, 10, 48, 34, 31, 67, 4, 82, 71, 14, 80, 16, 93, 69, 30, 29, 16, 42, 21, 11, 18, 56, 65, 1, 57, 94, 75, 90, 23, 19, 8, 30, 24, 96, 45, 96, 97, 82, 5, 52, 99, 81, 58, 70, 72, 91, 18]

In [0]:
print(type(random_rdd.collect()))

<class 'list'>

You should be very careful about using `collect()` on an RDD containing a very large dataset. When working on a cluster, the contents of an RDD will be split across the nodes in that cluster. By calling the `collect()` method, you are requesting that all of the elements in the RDD be collected onto the driver as an in-memory list. If the resulting list is too large to fit into the memory of the node running the driver, that node will likely crash killing your application. 

In a later lesson we will discuss how to using sampling and subsetting to explore the contents of a large RDD without collecting the entire RDD into memory on the driver.

## Descriptive Statistics

Spark provides several RDD methods for calculating descriptive statistics. These include the following methods: `count()`, `sum()`, `mean()`, `variance()`, `stdev()`, `min()`, and `max()`. Each of these methods represent an RDD action. Note that many of these actions can only be used on RDDs containing numerical values. 

We will use `random_rdd` to demonstrate the use of these actions in the cell below.

In [0]:
print('Count:   ', random_rdd.count())    # Number of elements
print('Sum:     ', random_rdd.sum())      # Sum of elements
print('Mean:    ', random_rdd.mean())     # Mean (or average)
print('Variance:', random_rdd.variance()) # Population Variance
print('Std Dev: ', random_rdd.stdev())    # Population Standard Deviation
print('Minimum: ', random_rdd.min())      # Minimum (smallest) element
print('Maximum: ', random_rdd.max())      # Maximum (largest) element

You can calculate the count, mean, standard deviation, min, and max of an RDD with a single call to the `stats()` action.

In [0]:
print(random_rdd.stats())

## Reading from a Text File

We can use the `textFile()` method to create an RDD from a text file stored in a local file system, as long as that file is in a location that is accessible by each of the nodes in the cluster. Each line in the text file will be stored as a single element of the resulting RDD and will be represented as a string within the new RDD.

In the cell below, we read the contents of a file named `datafile_01.txt`. This file contains a single line of text. That line contains several integer values separated by spaces. Notice that the resulting RDD contains only a single element.

In [0]:
temp_rdd_1 = sc.textFile('/FileStore/tables/datafile_01.txt')
print(temp_rdd_1.count())
print(temp_rdd_1.collect())

1
['5 7 4 7 8 2 5 6 2 7 5 3']

In the next cell, we read the contents of a file named `datafile_02.txt`. This file contains twelve lines of text. Each line contains a single character representing an integer value. Notice that the RDD created from this file contains twelve elements, and that each of the elements are read in to the RDD as strings.

In [0]:
temp_rdd_2 = sc.textFile('/FileStore/tables/datafile_02.txt')
print(temp_rdd_2.count())
print(temp_rdd_2.collect())

12
['5', '7', '4', '7', '8', '2', '5', '6', '2', '7', '5', '3']

As one last example of using `textFile()`, we will read the contents of a file containing the text of the novel "A Tale of Two Cities". We see that this file contains 15,797 lines of text.

In [0]:
totc = sc.textFile('/FileStore/tables/tale_of_two_cities.txt')
print(totc.count())

15797

We will now display the first 25 lines from this text file.

In [0]:
first25 = totc.collect()[:25]
#first25 = totc.take(25) 
# this is a better way to look at the first 25, I am only calculate the first 25 items that I need. versus the first one reads all the items in memory

for i, line in enumerate(first25):
    print(f'Line {i+1:02}:  {line}')

Line 01: A TALE OF TWO CITIES
Line 02: 
Line 03: A STORY OF THE FRENCH REVOLUTION
Line 04: 
Line 05: By Charles Dickens
Line 06: 
Line 07: Book the First--Recalled to Life
Line 08: 
Line 09: I. The Period
Line 10: 
Line 11: It was the best of times,
Line 12: it was the worst of times,
Line 13: it was the age of wisdom,
Line 14: it was the age of foolishness,
Line 15: it was the epoch of belief,
Line 16: it was the epoch of incredulity,
Line 17: it was the season of Light,
Line 18: it was the season of Darkness,
Line 19: it was the spring of hope,
Line 20: it was the winter of despair,
Line 21: we had everything before us,
Line 22: we had nothing before us,
Line 23: we were all going direct to Heaven,
Line 24: we were all going direct the other way--
Line 25: in short, the period was so far like the present period, that some of